In [3]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

import pandas as pd
import sqlite3
import hashlib
from sqlalchemy import text, inspect, MetaData, Table
from sqlalchemy.orm import sessionmaker
from app.main import getTables, Base, Account, Transaction, TransactionFinal, Load
from app.helpers.readCsv import openOneCsv, getAccountHistory
from app.helpers.sqlEngine import engine

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 1000)

Session = sessionmaker(bind=engine)
session = Session()

session.connection().execute(text("DELETE FROM loads"))
session.connection().execute(text("DELETE FROM transactions_all"))
session.connection().commit()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
accounts = getTables(engine)
session.rollback()
for index, account in accounts.iterrows():
    path = account['csvPath']
    print(path)
    query = f"SELECT * FROM Loads WHERE accountId = {account['id']};"
    print(query, account['id'])
    allFiles = getAccountHistory(path, account['id'], session)
    display(allFiles)

chaseChecking/
SELECT * FROM Loads WHERE accountId = 1; 1
reading Chase8115_Activity_20230418
 path=./db/rawTransactionHistory/chaseChecking//Chase8115_Activity_20230418.CSV
  matchingLoads.shape=(0, 4)
  load.id=None
  load.id=1 (post-commit)
reading Chase8115_Activity_20230425
 path=./db/rawTransactionHistory/chaseChecking//Chase8115_Activity_20230425.CSV
  matchingLoads.shape=(0, 4)
  load.id=None
  load.id=2 (post-commit)
reading Chase8115_Activity_20230419
 path=./db/rawTransactionHistory/chaseChecking//Chase8115_Activity_20230419.CSV
  matchingLoads.shape=(0, 4)
  load.id=None
  load.id=3 (post-commit)
reading Chase8115_Activity_20230423
 path=./db/rawTransactionHistory/chaseChecking//Chase8115_Activity_20230423.CSV
  matchingLoads.shape=(0, 4)
  load.id=None
  load.id=4 (post-commit)


/tmp/ipykernel_23038/3606625983.py:2: SAWarning: transaction already deassociated from connection
  session.rollback()


,details,description,amount,type,balance,checkOrSlipNumber,sha256,seenTime,filename,Post Date,loadID,Details,Description,Amount,Type,Balance,Check or Slip #
0,NaN,NaN,NaN,NaN,NaN,NaN,313ac6974c,2023-05-02 20:32:18.998062,./db/rawTransactionHistory/chaseChecking//Chase8115_Activity_20230425.CSV,2023-04-25,NaN,DEBIT,POS DEBIT TRADITIONAL TKD CENTER TAMPA FL,-198.00,MISC_DEBIT,,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,3db3a14875,2023-05-02 20:32:18.998062,./db/rawTransactionHistory/chaseChecking//Chase8115_Activity_20230425.CSV,2023-04-24,NaN,DEBIT,NON-CHASE ATM FEE-WITH,-3.00,FEE_TRANSACTION,3112.89,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,16ba5a0609,2023-05-02 20:32:18.998062,./db/rawTransactionHistory/chaseChecking//Chase8115_Activity_20230425.CSV,2023-04-24,NaN,DEBIT,NON-CHASE ATM WITHDRAW 665596 04/23500 ORANG,-24.00,ATM,3115.89,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,5b8a330ea4,2023-05-02 20:32:18.998062,./db/rawTransactionHistory/chaseChecking//Chase8115_Activity_20230425.CSV,2023-04-21,NaN,CREDIT,GOODWAYGROUPINC PAYROLL PPD ID: 2591583162,1701.48,ACH_CREDIT,3139.89,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,5b8a330ea4,2023-05-02 20:32:19.036027,./db/rawTransactionHistory/chaseChecking//Chase8115_Activity_20230423.CSV,2023-04-21,NaN,CREDIT,GOODWAYGROUPINC PAYROLL PPD ID: 2591583162,1701.48,ACH_CREDIT,3139.89,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,NaN,NaN,NaN,NaN,NaN,NaN,4cf92916c6,2023-05-02 20:32:19.020544,./db/rawTransactionHistory/chaseChecking//Chase8115_Activity_20230419.CSV,2021-04-19,NaN,DEBIT,WENDYS #0331 TAMPA FL 04/17,-13.17,DEBIT_CARD,1043.32,NaN
1297,DEBIT,NON-CHASE ATM WITHDRAW 925403 04/183615 WEST,-42.95,ATM,994.53,NaN,2097ca6cd4,2023-05-02 20:32:18.980437,./db/rawTransactionHistory/chaseChecking//Chase8115_Activity_20230418.CSV,2021-04-19,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1297,NaN,NaN,NaN,NaN,NaN,NaN,2097ca6cd4,2023-05-02 20:32:19.020544,./db/rawTransactionHistory/chaseChecking//Chase8115_Activity_20230419.CSV,2021-04-19,NaN,DEBIT,NON-CHASE ATM WITHDRAW 925403 04/183615 WEST,-42.95,ATM,994.53,NaN
1295,DEBIT,POPEYES 12619 CLEARWATER FL 04/18,-21.40,DEBIT_CARD,932.91,NaN,1e7a0516f8,2023-05-02 20:32:18.980437,./db/rawTransactionHistory/chaseChecking//Chase8115_Activity_20230418.CSV,2021-04-19,1.0,NaN,NaN,NaN,NaN,NaN,NaN


chaseFreedom/
SELECT * FROM Loads WHERE accountId = 2; 2
reading Chase4908_Activity20230101_20230418_20230418
 path=./db/rawTransactionHistory/chaseFreedom//Chase4908_Activity20230101_20230418_20230418.CSV
  matchingLoads.shape=(0, 4)
  load.id=None
  load.id=5 (post-commit)
reading Chase4908_Activity20221201_20230423_20230423
 path=./db/rawTransactionHistory/chaseFreedom//Chase4908_Activity20221201_20230423_20230423.CSV
  matchingLoads.shape=(0, 4)
  load.id=None
  load.id=6 (post-commit)
reading Chase4908_Activity20230425
 path=./db/rawTransactionHistory/chaseFreedom//Chase4908_Activity20230425.CSV
  matchingLoads.shape=(0, 4)
  load.id=None
  load.id=7 (post-commit)
reading Chase4908_Activity20230419
 path=./db/rawTransactionHistory/chaseFreedom//Chase4908_Activity20230419.CSV
  matchingLoads.shape=(0, 4)
  load.id=None
  load.id=8 (post-commit)


,Transaction Date,Post Date,description,Category,type,amount,Memo,sha256,seenTime,filename,loadID,Description,Type,Amount
1,2023-04-22,2023-04-24,NaN,Food & Drink,NaN,NaN,NaN,c82c3ff658,2023-05-02 20:32:19.100725,./db/rawTransactionHistory/chaseFreedom//Chase4908_Activity20230425.CSV,NaN,Lobos Coffee,Sale,-9.37
3,2023-04-23,2023-04-24,NaN,Shopping,NaN,NaN,NaN,c0875001d6,2023-05-02 20:32:19.100725,./db/rawTransactionHistory/chaseFreedom//Chase4908_Activity20230425.CSV,NaN,SQ *CARLA POMA,Sale,-27.69
4,2023-04-23,2023-04-24,NaN,Food & Drink,NaN,NaN,NaN,93e2967a0b,2023-05-02 20:32:19.100725,./db/rawTransactionHistory/chaseFreedom//Chase4908_Activity20230425.CSV,NaN,SQ *SKYEBIRD,Sale,-25.56
2,2023-04-22,2023-04-24,NaN,Travel,NaN,NaN,NaN,820f6f9255,2023-05-02 20:32:19.100725,./db/rawTransactionHistory/chaseFreedom//Chase4908_Activity20230425.CSV,NaN,parkreceipts.com T,Sale,-6.00
0,2023-04-24,2023-04-24,NaN,Fees & Adjustments,NaN,NaN,NaN,660a3b899a,2023-05-02 20:32:19.100725,./db/rawTransactionHistory/chaseFreedom//Chase4908_Activity20230425.CSV,NaN,REDEMPTION CREDIT,Adjustment,1677.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,2022-12-01,2022-12-02,NaN,Bills & Utilities,NaN,NaN,NaN,05b77bf16d,2023-05-02 20:32:19.090627,./db/rawTransactionHistory/chaseFreedom//Chase4908_Activity20221201_20230423_20230423.CSV,NaN,DREAMHOST,Sale,-32.95
493,2022-11-30,2022-12-01,NaN,Groceries,NaN,NaN,NaN,741a742bdf,2023-05-02 20:32:19.090627,./db/rawTransactionHistory/chaseFreedom//Chase4908_Activity20221201_20230423_20230423.CSV,NaN,PUBLIX #1506,Sale,-87.70
492,2022-11-30,2022-12-01,NaN,Food & Drink,NaN,NaN,NaN,1f755e2328,2023-05-02 20:32:19.090627,./db/rawTransactionHistory/chaseFreedom//Chase4908_Activity20221201_20230423_20230423.CSV,NaN,GUTHRIES CHICKEN - TAMPA,Sale,-10.64
494,2022-11-30,2022-12-01,NaN,Food & Drink,NaN,NaN,NaN,1f755e2328,2023-05-02 20:32:19.090627,./db/rawTransactionHistory/chaseFreedom//Chase4908_Activity20221201_20230423_20230423.CSV,NaN,GUTHRIES CHICKEN - TAMPA,Sale,-10.64


In [5]:
inspector = inspect(engine)
tables = inspector.get_table_names()
for table in tables:
    print(table)

metadata = MetaData()
metadata.reflect(bind=engine)

def listCols(tablename):
    table = metadata.tables[tablename]
    print(tablename)
    for column in table.columns:
        print(f"   {column}")

listCols('accounts')
listCols('loads')


pd.read_sql(sql=text("select * from loads"), con=engine.connect())   

    

accounts
loads
transactions_all
transactions_final
accounts
   accounts.id
   accounts.name
   accounts.accountNumber
   accounts.interestRate
   accounts.statementDate
   accounts.paymentDueDate
   accounts.csvPath
loads
   loads.id
   loads.accountId
   loads.fullFilePath
   loads.dateTime


,id,accountId,fullFilePath,dateTime
0,1,1,./db/rawTransactionHistory/chaseChecking//Chase8115_Activity_20230418.CSV,2023-05-02 23:46:11.159076
1,2,1,./db/rawTransactionHistory/chaseChecking//Chase8115_Activity_20230425.CSV,2023-05-02 23:46:11.167754
2,3,1,./db/rawTransactionHistory/chaseChecking//Chase8115_Activity_20230419.CSV,2023-05-02 23:46:11.188199
3,4,1,./db/rawTransactionHistory/chaseChecking//Chase8115_Activity_20230423.CSV,2023-05-02 23:46:11.201471
4,5,2,./db/rawTransactionHistory/chaseFreedom//Chase4908_Activity20230101_20230418_20230418.CSV,2023-05-02 23:46:11.234335
5,6,2,./db/rawTransactionHistory/chaseFreedom//Chase4908_Activity20221201_20230423_20230423.CSV,2023-05-02 23:46:11.248919
6,7,2,./db/rawTransactionHistory/chaseFreedom//Chase4908_Activity20230425.CSV,2023-05-02 23:46:11.279385
7,8,2,./db/rawTransactionHistory/chaseFreedom//Chase4908_Activity20230419.CSV,2023-05-02 23:46:11.289648


In [5]:
session.connection().execute(text("DELETE FROM loads"))
session.connection().execute(text("DELETE FROM transactions_all"))
session.connection().commit()